# 1. Info

This notebook contains the code required to transform the raw data to the enriched data needed to predict the result.

# 2. Reading the new data

In [1]:
# import libraries
from functions import away_result, home_result, get_points, get_result
import pandas as pd

In [2]:
# read historical data
historical_data = pd.read_csv('../data/enriched_data/premier_league.csv')

In [3]:
# read data to predict
predict_data = pd.read_csv('../data/predict_data.csv')

In [4]:
# normalize team names
predict_data['HomeTeam'] = predict_data['HomeTeam'].str.replace("'","").str.replace(" ","_")
predict_data['AwayTeam'] = predict_data['AwayTeam'].str.replace("'","").str.replace(" ","_")

In [5]:
# read teams dict 
teams_dict_df = pd.read_csv('../data/teams_dict.csv')

In [6]:
# create the dict
team_dict = {}
for tuple in teams_dict_df.itertuples():
    team_dict.update({tuple.Team:tuple.index_team})

predict_data['HomeTeam'] = predict_data['HomeTeam'].replace(team_dict)
predict_data['AwayTeam'] = predict_data['AwayTeam'].replace(team_dict)

In [7]:
# adjust column types
for column in predict_data.loc[:, predict_data.dtypes == float].columns:
    predict_data[column] = predict_data[column].astype('int')

In [8]:
team_data = {
    'Date':[],
    'Team':[],
    'Opponent':[],
    'season':[],
    'Home':[]
}

In [9]:
# convert data to predict in the same format as historical data
for tuple in predict_data.itertuples():
    # home
    team_data['Date'].append(tuple.Date)
    team_data['Team'].append(tuple.HomeTeam)
    team_data['Opponent'].append(tuple.AwayTeam)
    team_data['season'].append(tuple.season)
    team_data['Home'].append(1)

    # away
    team_data['Date'].append(tuple.Date)
    team_data['Team'].append(tuple.AwayTeam)
    team_data['Opponent'].append(tuple.HomeTeam)
    team_data['season'].append(tuple.season)
    team_data['Home'].append(0)

In [10]:
predict_df = pd.DataFrame(data=team_data)

In [11]:
# concat historical and predict data into one df
df = pd.concat([historical_data,predict_df])

In [12]:
# sort by date
df['Date'] = pd.to_datetime(df['Date'], format="%Y-%m-%d")
sorted_data = df.sort_values(by=['season','Date']).reset_index(drop=True)

# get the current season
season = sorted_data.query("win_rate.isna()").reset_index()['season'][0]

In [13]:
# add 'ftg_scored_total','htg_scored_total','points','goal_difference'

season_data_teams = sorted_data.query(f"season == '{season}'").reset_index().copy()

# going only into the null rows, because are the ones that needs to be update
for tuple in season_data_teams.query("win_rate.isna()").itertuples():

    # every itteration will require to create a new season_data dataFrame to
    # have the most updated data to find the correct values
    season_data = sorted_data.query(f"season == '{season}'").query(f"Team == {tuple.Team}").reset_index().copy()

    # check if is the first round and set everything to 0
    if season_data.query("FTG_scored_Total.notna()").shape[0] == 0:
        sorted_data.loc[tuple.index,'FTG_scored_Total'] = 0
        sorted_data.loc[tuple.index,'FTG_received_Total'] = 0
        sorted_data.loc[tuple.index,'HTG_scored_Total'] = 0
        sorted_data.loc[tuple.index,'points'] = 0
        sorted_data.loc[tuple.index,'goal_difference'] = 0

    # when is not the first round, check the last information available and perform the operations    
    else:
        GF_last_match = season_data.query("FTG_scored.notna()").sort_values(by=['Date'], ascending=False).reset_index()['FTG_scored'][0]
        GA_last_match = season_data.query("FTG_received.notna()").sort_values(by=['Date'], ascending=False).reset_index()['FTG_received'][0]
        Result = get_result(GF_last_match,GA_last_match)

        GF_last_match_HT = season_data.query("HTG_scored.notna()").sort_values(by=['Date'], ascending=False).reset_index()['HTG_scored'][0]
        
        # add points
        sorted_data.loc[tuple.index,'points'] = season_data.query("points.notna()").sort_values(by=['Date'], ascending=False).reset_index()['points'][0] + get_points(Result)
        
        # add goals_scored FT
        sorted_data.loc[tuple.index,'FTG_scored_Total'] = season_data.query("FTG_scored_Total.notna()").sort_values(by=['Date'], ascending=False).reset_index()['FTG_scored_Total'][0] + int(GF_last_match)

        # add goals_received FT
        sorted_data.loc[tuple.index,'FTG_received_Total'] = season_data.query("FTG_received_Total.notna()").sort_values(by=['Date'], ascending=False).reset_index()['FTG_received_Total'][0] + int(GA_last_match)

        # add goals_scored HT
        sorted_data.loc[tuple.index,'HTG_scored_Total'] = season_data.query("HTG_scored_Total.notna()").sort_values(by=['Date'], ascending=False).reset_index()['HTG_scored_Total'][0] + int(GF_last_match_HT)

        sorted_data.loc[tuple.index,'goal_difference'] = sorted_data.loc[tuple.index,'FTG_scored_Total'] - sorted_data.loc[tuple.index,'FTG_received_Total']

In [14]:
# add 'position'

season_data = sorted_data.query(f"season == '{season}'").reset_index().copy()

for tuple in season_data.query("win_rate.isna()").itertuples():

    if season_data.query("win_rate.notna()").shape[0] == 0:
        sorted_data.loc[tuple.index,'position'] = 0
    
    else:
        # the season dataFrame will be filtered to get the past data, drop duplicates
        # sort by points and goals and finally merge the output with the data from the
        # current fixture
        past_data = season_data.sort_values(by=['Date']).copy()
        unique_teams_data = past_data.drop_duplicates(
            subset=['Team'], 
            keep='last'
        )
        position_data = unique_teams_data.sort_values(
            by=['points','goal_difference','FTG_scored_Total','FTG_received_Total'],
            ascending=False
        ).reset_index(drop=True)

        current_position_data = position_data[['Team']].reset_index()
        current_position_data.columns = ['current_position','Team']
        current_position_data['current_position'] = current_position_data['current_position']+1

        sorted_data.loc[tuple.index,'position'] = current_position_data.query(f"Team == {tuple.Team}").reset_index()['current_position'][0]

In [15]:
# add 'win_rate','mooving_win_rate','mooving_goals_scored'

# this new column is because when creating the win rate, the only result that should sum must be the win.
sorted_data['Win'] = sorted_data['FT_Result'].replace({2:0})

idx_delete = []

for season in sorted_data['season'].unique().tolist():
    # get data season
    data_season = sorted_data.query(f"season == '{season}'")

    # iterate by team
    for team in data_season.query("win_rate.isna()")['Team'].unique().tolist():
        # get data by team
        data_team = data_season.query(f"Team == {team}").reset_index().rename(columns={'index':'old_idx'})


        # get the win_rate, mooving win_rate and mooving_goals
        for idx, row in data_team.iterrows():
            # condition to avoid out of index mistake
            if idx + 1 <= len(data_team['old_idx']):

                # win_rate
                total_wins = int(data_team.query(f"old_idx < {row['old_idx']}")['Win'].sum())
                if total_wins == 0:
                    sorted_data.loc[data_team.loc[idx,'old_idx'],'win_rate'] = 0
                else:
                    sorted_data.loc[data_team.loc[idx,'old_idx'],'win_rate'] = total_wins / idx

                # mooving_win_rate
                # get the index list filter
                if idx < 5:
                    index_list = [idx_number for idx_number in range(0,idx+1)]
                else:
                    index_list = [idx_number for idx_number in range(idx-4,idx+1)]

                total_wins = int(data_team.query(f"index in {index_list}")['Win'].sum())
                if total_wins == 0:
                    sorted_data.loc[data_team.loc[idx,'old_idx'],'mooving_win_rate'] = 0
                elif idx == 0:
                    sorted_data.loc[data_team.loc[idx,'old_idx'],'mooving_win_rate'] = 0
                else:
                    sorted_data.loc[data_team.loc[idx,'old_idx'],'mooving_win_rate'] = total_wins / 5
                
                # mooving goals scored
                total_goals_scored = int(data_team.query(f"index in {index_list}")['FTG_scored_Total'].sum())
                if total_goals_scored == 0:
                    sorted_data.loc[data_team.loc[idx,'old_idx'],'mooving_goals_scored'] = 0
                elif idx == 0:
                    sorted_data.loc[data_team.loc[idx,'old_idx'],'mooving_goals_scored'] = 0
                else:
                    sorted_data.loc[data_team.loc[idx,'old_idx'],'mooving_goals_scored'] = total_goals_scored / 5

                # mooving goals received
                total_goals_received = int(data_team.query(f"index in {index_list}")['FTG_received_Total'].sum())
                if total_goals_received == 0:
                    sorted_data.loc[data_team.loc[idx,'old_idx'],'mooving_goals_received'] = 0
                elif idx == 0:
                    sorted_data.loc[data_team.loc[idx,'old_idx'],'mooving_goals_received'] = 0
                else:
                    sorted_data.loc[data_team.loc[idx,'old_idx'],'mooving_goals_received'] = total_goals_received / 5

                # index of rows to be delete
                if idx < 5:
                    idx_delete.append(data_team.loc[idx,'old_idx'])


In [16]:
sorted_data.columns = sorted_data.columns.str.lower().str.replace(' ', '_')

In [17]:
data_to_predict = sorted_data.query("win.isna()")[['ftg_scored_total','htg_scored_total','points','goal_difference','position','win_rate','mooving_win_rate','mooving_goals_scored']]

In [18]:
# adjust column types
for column in data_to_predict.loc[:, data_to_predict.dtypes == float].columns:
    if column not in ['mooving_win_rate','mooving_goals_scored','win_rate']:
        data_to_predict[column] = data_to_predict[column].astype('int')

In [19]:
data_to_predict.to_csv('../data/to_predict/premier_league_001.csv', index=False)

End of notebook